<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata


engine = create_engine(userdata.get("supabase"), client_encoding='utf8', )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [74]:
SQL = """
SELECT
    platform,
	query,
	count(*) AS cnt,
	sum(count(query)) over(PARTITION BY platform) AS platform_total,
	sum(count(query)) over(PARTITION BY query) AS query_count_total

FROM
	vizro.yandex_data yd
GROUP BY
	platform,
	query
"""

df = select(SQL)

In [75]:
min_query_cnt = 50
df = df.query("query_count_total >= @min_query_cnt")

In [76]:
df_pivoted = df.pivot(index='query', columns=["platform"], values=["cnt", "platform_total"],)

In [77]:
df_pivoted.fillna(value={('cnt', 'touch'):0, ('cnt', 'desktop'):0, ('platform_total', 'desktop'): df_pivoted[('platform_total', 'desktop')].max(), ('platform_total', 'touch'):df_pivoted[('platform_total', 'touch')].max()}, inplace=True)

In [38]:
from statsmodels.stats.proportion import proportions_chisquare

In [87]:
from statsmodels.stats.proportion import proportions_chisquare

def proportions_chi2(df: pd.DataFrame):

  _, pval, _ = proportions_chisquare(
  count=[df[('cnt', 'desktop')], df[('cnt', 'touch')]],
  nobs=[df[('platform_total', 'desktop')], df[('platform_total', 'touch')]])

  return pval

In [88]:
df_pivoted["pval"] = df_pivoted.apply(lambda x: proportions_chi2(x), axis=1)

In [89]:
df_pivoted

cnt        platform_total                    pval
platform                desktop  touch        desktop     touch              
query                                                                        
02 аниме                   34.0   30.0       411566.0  792258.0  1.403780e-03
1 сентября                277.0  997.0       411566.0  792258.0  7.248046e-21
1 сентября 2021            19.0   71.0       411566.0  792258.0  8.908993e-03
1 сентября день знаний     69.0  137.0       411566.0  792258.0  8.338784e-01
1 сентября картинки       223.0  653.0       411566.0  792258.0  5.030819e-08
...                         ...    ...            ...       ...           ...
ячмень на глазу лечение    12.0   56.0       411566.0  792258.0  4.031404e-03
яэ мико                    48.0   43.0       411566.0  792258.0  1.895721e-04
ёжики                     107.0  123.0       411566.0  792258.0  8.023864e-05
سكس                         6.0   77.0       411566.0  792258.0  2.241255e-07
视频                          0.0   69.0       411566.0  792258.0  2.134905e-09

[5088 rows x 5 columns]